https://haystack.deepset.ai/tutorials/train-dpr

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install library

In [ ]:
!pip install farm-haystack --find-links=https://download.pytorch.org/whl/torch_stable.html
!pip install grpcio-tools==1.34.1
!pip install git+https://github.com/deepset-ai/haystack.git
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached transformers-4.7.0-py3-none-any.whl (2.5 MB)
  Using cached huggingface_hub-0.0.8-py3-none-any.whl (34 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.2.1
    Uninstalling huggingface-hub-0.2.1:
      Successfully uninstalled huggingface-hub-0.2.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.13.0
    Uninstalling transformers-4.13.0:
      Successfully uninstalled transformers-4.13.0
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-3bp_me5g
  Running command git clone -q https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-3bp_me5g
  Using cached transformers-4.13.0-py3-none-any.whl (3.3 MB)
  Using cached huggingface_hub-0.2.1-py3-none-any.whl (61 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.8
    Uninstalling huggingface-hub-0.0.8

# Import Library

In [ ]:
import pandas as pd
from haystack.utils import clean_wiki_text, convert_files_to_dicts, fetch_archive_from_http, print_answers
from haystack.nodes import FARMReader, TransformersReader, TfidfRetriever, DensePassageRetriever
from haystack.pipelines import ExtractiveQAPipeline
from haystack.document_stores import InMemoryDocumentStore
from tqdm import tqdm

import json
from pathlib import Path
from typing import Dict, Iterator, Tuple, List, Union

/usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)


In [ ]:
# Load library for preprocess
from haystack.nodes import PreProcessor
from haystack.utils import convert_files_to_dicts 

In [ ]:
path="/content/drive/MyDrive/Data/Squad2.0"

## Reader 1: XLM-Roberta

In [ ]:
path_model ="/content/drive/MyDrive/Data/Squad2.0/my_model"
reader = FARMReader(model_name_or_path = path_model)

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Model found locally at /content/drive/MyDrive/Data/Squad2.0/my_model
INFO - haystack.modeling.model.language_model -  Loaded /content/drive/MyDrive/Data/Squad2.0/my_model
INFO - haystack.modeling.model.adaptive_model -  Found files for loading 1 prediction heads
WARNING - haystack.modeling.model.prediction_head -  Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"training": false, "num_labels": 2, "ph_output_type": "per_token_squad", "model_type": "span_classification", "label_tensor_name": "question_answering_label_ids", "label_list": ["start_token", "end_token"], "metric": "squad", "name": "QuestionAnsweringHead"}
INFO - haystack.modeling.model.prediction_head 

In [ ]:
# Evaluate Reader on its own
#reader_eval_results = reader.eval(document_store=document_store, device=device, label_index=label_index, doc_index=doc_index)
# Evaluation of Reader can also be done directly on a SQuAD-formatted file without passing the data to Elasticsearch
reader_eval_results = reader.eval_on_file(path, "new_train.json")

## Reader Top-N-Accuracy is the proportion of predicted answers that match with their corresponding correct answer
print("Reader Top-N-Accuracy:", reader_eval_results["top_n_accuracy"])
## Reader Exact Match is the proportion of questions where the predicted answer is exactly the same as the correct answer
print("Reader Exact Match:", reader_eval_results["EM"])
## Reader F1-Score is the average overlap between the predicted answers and the correct answers
print("Reader F1-Score:", reader_eval_results["f1"])

INFO - haystack.modeling.data_handler.data_silo -  
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
INFO - haystack.modeling.data_handler.data_silo -  LOADING TRAIN DATA
INFO - haystack.modeling.data_handler.data_silo -  ==================
INFO - haystack.modeling.data_handler.data_silo -  No train set is being loaded
INFO - haystack.modeling.data_handler.data_silo -  
INFO - haystack.modeling.data_handler.data_silo -  LOADING DEV DATA
INFO - haystack.modeling.data_handler.data_silo -  =================
INFO - haystack.modeling.data_handler.data_silo -  No dev set is being loaded
INFO - haystack.modeling.data_handler.data_silo -  
INFO - haystack.modeling.data_handler.data_silo -  LOADING TEST DATA
INFO - haystack.modeling.data_handler.data_silo -  =================
INFO - haystack.modeling.data_handler.data_silo -  Loading test set from: /content/drive/My

Reader Top-N-Accuracy: 0.9914957747995048
Reader Exact Match: 0.8123688034878088
Reader F1-Score: 0.9448299862794828


In [ ]:
# Evaluate Reader on its own
#reader_eval_results = reader.eval(document_store=document_store, device=device, label_index=label_index, doc_index=doc_index)
# Evaluation of Reader can also be done directly on a SQuAD-formatted file without passing the data to Elasticsearch
reader_eval_results = reader.eval_on_file(path, "new_dev.json")

## Reader Top-N-Accuracy is the proportion of predicted answers that match with their corresponding correct answer
print("Reader Top-N-Accuracy:", reader_eval_results["top_n_accuracy"])
## Reader Exact Match is the proportion of questions where the predicted answer is exactly the same as the correct answer
print("Reader Exact Match:", reader_eval_results["EM"])
## Reader F1-Score is the average overlap between the predicted answers and the correct answers
print("Reader F1-Score:", reader_eval_results["f1"])

INFO - haystack.modeling.data_handler.data_silo -  
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
INFO - haystack.modeling.data_handler.data_silo -  LOADING TRAIN DATA
INFO - haystack.modeling.data_handler.data_silo -  ==================
INFO - haystack.modeling.data_handler.data_silo -  No train set is being loaded
INFO - haystack.modeling.data_handler.data_silo -  
INFO - haystack.modeling.data_handler.data_silo -  LOADING DEV DATA
INFO - haystack.modeling.data_handler.data_silo -  =================
INFO - haystack.modeling.data_handler.data_silo -  No dev set is being loaded
INFO - haystack.modeling.data_handler.data_silo -  
INFO - haystack.modeling.data_handler.data_silo -  LOADING TEST DATA
INFO - haystack.modeling.data_handler.data_silo -  =================
INFO - haystack.modeling.data_handler.data_silo -  Loading test set from: /content/drive/My

Reader Top-N-Accuracy: 0.9050328227571116
Reader Exact Match: 0.6201312910284464
Reader F1-Score: 0.8025070187608059


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
# Evaluate Reader on its own
#reader_eval_results = reader.eval(document_store=document_store, device=device, label_index=label_index, doc_index=doc_index)
# Evaluation of Reader can also be done directly on a SQuAD-formatted file without passing the data to Elasticsearch
reader_eval_results = reader.eval_on_file(path, "new_test.json")

## Reader Top-N-Accuracy is the proportion of predicted answers that match with their corresponding correct answer
print("Reader Top-N-Accuracy:", reader_eval_results["top_n_accuracy"])
## Reader Exact Match is the proportion of questions where the predicted answer is exactly the same as the correct answer
print("Reader Exact Match:", reader_eval_results["EM"])
## Reader F1-Score is the average overlap between the predicted answers and the correct answers
print("Reader F1-Score:", reader_eval_results["f1"])

INFO - haystack.modeling.data_handler.data_silo -  
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
INFO - haystack.modeling.data_handler.data_silo -  LOADING TRAIN DATA
INFO - haystack.modeling.data_handler.data_silo -  ==================
INFO - haystack.modeling.data_handler.data_silo -  No train set is being loaded
INFO - haystack.modeling.data_handler.data_silo -  
INFO - haystack.modeling.data_handler.data_silo -  LOADING DEV DATA
INFO - haystack.modeling.data_handler.data_silo -  =================
INFO - haystack.modeling.data_handler.data_silo -  No dev set is being loaded
INFO - haystack.modeling.data_handler.data_silo -  
INFO - haystack.modeling.data_handler.data_silo -  LOADING TEST DATA
INFO - haystack.modeling.data_handler.data_silo -  =================
INFO - haystack.modeling.data_handler.data_silo -  Loading test set from: /content/drive/My

Reader Top-N-Accuracy: 0.9135746606334841
Reader Exact Match: 0.6221719457013575
Reader F1-Score: 0.8045488833214228


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


# Reader 2: Bert-Base-Multilingual

In [ ]:
path_model ="/content/drive/MyDrive/Data/Squad2.0/my_model2"
reader = FARMReader(model_name_or_path = path_model)

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Model found locally at /content/drive/MyDrive/Data/Squad2.0/my_model2
INFO - haystack.modeling.model.language_model -  Loaded /content/drive/MyDrive/Data/Squad2.0/my_model2
INFO - haystack.modeling.model.adaptive_model -  Found files for loading 1 prediction heads
WARNING - haystack.modeling.model.prediction_head -  Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"training": false, "num_labels": 2, "ph_output_type": "per_token_squad", "model_type": "span_classification", "label_tensor_name": "question_answering_label_ids", "label_list": ["start_token", "end_token"], "metric": "squad", "name": "QuestionAnsweringHead"}
INFO - haystack.modeling.model.prediction_hea

In [ ]:
# Evaluate Reader on its own
#reader_eval_results = reader.eval(document_store=document_store, device=device, label_index=label_index, doc_index=doc_index)
# Evaluation of Reader can also be done directly on a SQuAD-formatted file without passing the data to Elasticsearch
reader_eval_results = reader.eval_on_file(path, "new_train.json")

## Reader Top-N-Accuracy is the proportion of predicted answers that match with their corresponding correct answer
print("Reader Top-N-Accuracy:", reader_eval_results["top_n_accuracy"])
## Reader Exact Match is the proportion of questions where the predicted answer is exactly the same as the correct answer
print("Reader Exact Match:", reader_eval_results["EM"])
## Reader F1-Score is the average overlap between the predicted answers and the correct answers
print("Reader F1-Score:", reader_eval_results["f1"])

INFO - haystack.modeling.data_handler.data_silo -  
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
INFO - haystack.modeling.data_handler.data_silo -  LOADING TRAIN DATA
INFO - haystack.modeling.data_handler.data_silo -  ==================
INFO - haystack.modeling.data_handler.data_silo -  No train set is being loaded
INFO - haystack.modeling.data_handler.data_silo -  
INFO - haystack.modeling.data_handler.data_silo -  LOADING DEV DATA
INFO - haystack.modeling.data_handler.data_silo -  =================
INFO - haystack.modeling.data_handler.data_silo -  No dev set is being loaded
INFO - haystack.modeling.data_handler.data_silo -  
INFO - haystack.modeling.data_handler.data_silo -  LOADING TEST DATA
INFO - haystack.modeling.data_handler.data_silo -  =================
INFO - haystack.modeling.data_handler.data_silo -  Loading test set from: /content/drive/My

Reader Top-N-Accuracy: 0.981968889606545
Reader Exact Match: 0.7901394047042359
Reader F1-Score: 0.9228667095462856


In [ ]:
# Evaluate Reader on its own
#reader_eval_results = reader.eval(document_store=document_store, device=device, label_index=label_index, doc_index=doc_index)
# Evaluation of Reader can also be done directly on a SQuAD-formatted file without passing the data to Elasticsearch
reader_eval_results = reader.eval_on_file(path, "new_dev.json")

## Reader Top-N-Accuracy is the proportion of predicted answers that match with their corresponding correct answer
print("Reader Top-N-Accuracy:", reader_eval_results["top_n_accuracy"])
## Reader Exact Match is the proportion of questions where the predicted answer is exactly the same as the correct answer
print("Reader Exact Match:", reader_eval_results["EM"])
## Reader F1-Score is the average overlap between the predicted answers and the correct answers
print("Reader F1-Score:", reader_eval_results["f1"])

INFO - haystack.modeling.data_handler.data_silo -  
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
INFO - haystack.modeling.data_handler.data_silo -  LOADING TRAIN DATA
INFO - haystack.modeling.data_handler.data_silo -  ==================
INFO - haystack.modeling.data_handler.data_silo -  No train set is being loaded
INFO - haystack.modeling.data_handler.data_silo -  
INFO - haystack.modeling.data_handler.data_silo -  LOADING DEV DATA
INFO - haystack.modeling.data_handler.data_silo -  =================
INFO - haystack.modeling.data_handler.data_silo -  No dev set is being loaded
INFO - haystack.modeling.data_handler.data_silo -  
INFO - haystack.modeling.data_handler.data_silo -  LOADING TEST DATA
INFO - haystack.modeling.data_handler.data_silo -  =================
INFO - haystack.modeling.data_handler.data_silo -  Loading test set from: /content/drive/My

Reader Top-N-Accuracy: 0.8682713347921225
Reader Exact Match: 0.5413566739606127
Reader F1-Score: 0.7332237854778118


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
# Evaluate Reader on its own
#reader_eval_results = reader.eval(document_store=document_store, device=device, label_index=label_index, doc_index=doc_index)
# Evaluation of Reader can also be done directly on a SQuAD-formatted file without passing the data to Elasticsearch
reader_eval_results = reader.eval_on_file(path, "new_test.json")

## Reader Top-N-Accuracy is the proportion of predicted answers that match with their corresponding correct answer
print("Reader Top-N-Accuracy:", reader_eval_results["top_n_accuracy"])
## Reader Exact Match is the proportion of questions where the predicted answer is exactly the same as the correct answer
print("Reader Exact Match:", reader_eval_results["EM"])
## Reader F1-Score is the average overlap between the predicted answers and the correct answers
print("Reader F1-Score:", reader_eval_results["f1"])

INFO - haystack.modeling.data_handler.data_silo -  
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
INFO - haystack.modeling.data_handler.data_silo -  LOADING TRAIN DATA
INFO - haystack.modeling.data_handler.data_silo -  ==================
INFO - haystack.modeling.data_handler.data_silo -  No train set is being loaded
INFO - haystack.modeling.data_handler.data_silo -  
INFO - haystack.modeling.data_handler.data_silo -  LOADING DEV DATA
INFO - haystack.modeling.data_handler.data_silo -  =================
INFO - haystack.modeling.data_handler.data_silo -  No dev set is being loaded
INFO - haystack.modeling.data_handler.data_silo -  
INFO - haystack.modeling.data_handler.data_silo -  LOADING TEST DATA
INFO - haystack.modeling.data_handler.data_silo -  =================
INFO - haystack.modeling.data_handler.data_silo -  Loading test set from: /content/drive/My

Reader Top-N-Accuracy: 0.8832579185520362
Reader Exact Match: 0.5276018099547511
Reader F1-Score: 0.7337224311440262


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
